In [45]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [46]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Portfolio Analysis").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [56]:
from pyspark.sql.functions import *

stock_lst = ["AAPL","ABBV","ABT","ADBE","ADP","AMGN","AMT","AMZN","AROW","ARTNA","AXP","AZN","BA", \
             "BABA","BAC","BCAL","BHP","BKNG","BTI","CAT","CECO","CHEF","CL","CMRX","COCO","CRM", \
             "CRML","CSCO","CTOS", "CVX","DEO","DIS","ELLO","EQIX","ETN","FAT","GCMG","GE","GOOGL", \
             "GRC","GS","GWRS","HIMX","HIPO","HON","IBM","ICHR","ISRG","JNJ","JPM","KLG","KMB","KO", \
             "LIN","LLY","LMT","MA","MCD","MDLZ","META","MS","MSFT","MXL","NFLX","NKE","NSPR","NTES", \
             "NVDA","ORCL","PCYO","PDD","PFE","PG","PLCE","PM","PNNT","PNST","RIO","SAP","SIRI","SNY", \
             "SPOT","SRG","SUP","TCI","TH","TJX","TM","TSLA", "TSM","UAN","UL","UNH","UNP","V","VZ","WFC", \
             "WMG","WMT","YTRA"]

stockdata =spark.sql("select * from stocks")

stockdata = stockdata.withColumn("date",to_date(stockdata.date,"yyyy-MM-dd"))


df=stockdata.groupBy("date").pivot("symbol", stock_lst).sum("adjClose")

#df = df.withColumnRenamed('BRK.B', 'BRK')
df.show()


filepath='file:////home/talentum/shared/Project/Project_codes/source/SP500.csv'

stock_data1=spark.read.option("header",True).csv(filepath)
#stock_data1.show()

stock_data1 = stock_data1.withColumn("date",to_date(stock_data1.Date,"MM/dd/yyyy"))

sp_500_df=stock_data1.select("date","Close")
sp_500_df=sp_500_df.withColumnRenamed("Close","SP500")
sp_500_df.show()

+----------+------+------+------+----------+------+------+------+------+-----+-----+------+-----+----------+------+-----+---------+-----+-------+-----+------+-----+-----+-----+------+---------+------+-----+-----+----+------+------+------+---------+------+------+-----+----+------+------+-----+------+-----+-----+---------+------+------+---------+------+------+------+-----+------+-----+------+------+------+------+------+-----+------+-----+------+-----+----------+------+-----+------+------+------+---------+----------+-----+------+---------+------+----+-----+-----+------+----+-----+----------+---------+----+---------+------+------+------+------+------+-----+-----+------+------+------+-----+-----+-----+-----+-----+
|      date|  AAPL|  ABBV|   ABT|      ADBE|   ADP|  AMGN|   AMT|  AMZN| AROW|ARTNA|   AXP|  AZN|        BA|  BABA|  BAC|     BCAL|  BHP|   BKNG|  BTI|   CAT| CECO| CHEF|   CL|  CMRX|     COCO|   CRM| CRML| CSCO|CTOS|   CVX|   DEO|   DIS|     ELLO|  EQIX|   ETN|  FAT|GCMG|    GE| 

In [57]:
#Creating returns Dataframe

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df_sp500=df.join(sp_500_df,"date")
#return_df_sp500.show()
my_window = Window.partitionBy().orderBy("date")
for col_name in return_df_sp500.columns:
    if col_name!='date':
     return_df = return_df_sp500.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df_sp500.columns:
    if c!='date':
        return_df_sp500 = return_df_sp500.withColumn(c, F.round(c, 6))

return_df_sp500=return_df_sp500.dropna()
return_df_sp500=return_df_sp500.drop('date')
return_df_sp500.show()
return_df=return_df_sp500.drop("SP500")
return_df.show()

+------+------+------+------+------+------+------+------+-----+-----+------+-----+------+-----+-----+-----+-----+-------+-----+------+-----+-----+------+-------+-----+------+-----+-----+----+------+------+------+-----+------+------+----+-----+------+------+-----+------+-----+----+-----+------+------+-----+-------+------+------+-----+------+-----+------+------+------+------+------+-----+------+------+------+-----+------+-----+----+------+------+------+-----+------+-----+------+-----+------+----+----+-----+------+----+-----+------+----+----+-----+-----+------+------+------+------+-----+-----+------+------+------+-----+-----+-----+-----+----+-------+
|  AAPL|  ABBV|   ABT|  ADBE|   ADP|  AMGN|   AMT|  AMZN| AROW|ARTNA|   AXP|  AZN|    BA| BABA|  BAC| BCAL|  BHP|   BKNG|  BTI|   CAT| CECO| CHEF|    CL|   CMRX| COCO|   CRM| CRML| CSCO|CTOS|   CVX|   DEO|   DIS| ELLO|  EQIX|   ETN| FAT| GCMG|    GE| GOOGL|  GRC|    GS| GWRS|HIMX| HIPO|   HON|   IBM| ICHR|   ISRG|   JNJ|   JPM|  KLG|   KMB|  

+------+------+------+------+------+------+------+------+-----+-----+------+-----+------+-----+-----+-----+-----+-------+-----+------+-----+-----+------+-------+-----+------+-----+-----+----+------+------+------+-----+------+------+----+-----+------+------+-----+------+-----+----+-----+------+------+-----+-------+------+------+-----+------+-----+------+------+------+------+------+-----+------+------+------+-----+------+-----+----+------+------+------+-----+------+-----+------+-----+------+----+----+-----+------+----+-----+------+----+----+-----+-----+------+------+------+------+-----+-----+------+------+------+-----+-----+-----+-----+----+
|  AAPL|  ABBV|   ABT|  ADBE|   ADP|  AMGN|   AMT|  AMZN| AROW|ARTNA|   AXP|  AZN|    BA| BABA|  BAC| BCAL|  BHP|   BKNG|  BTI|   CAT| CECO| CHEF|    CL|   CMRX| COCO|   CRM| CRML| CSCO|CTOS|   CVX|   DEO|   DIS| ELLO|  EQIX|   ETN| FAT| GCMG|    GE| GOOGL|  GRC|    GS| GWRS|HIMX| HIPO|   HON|   IBM| ICHR|   ISRG|   JNJ|   JPM|  KLG|   KMB|   KO|   L

In [5]:
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [6]:
def stocks_correlation(stocks_df,cols):
    # convert to vector column first
    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=cols, outputCol=vector_col)
    stocks_df_vector = assembler.transform(stocks_df).select(vector_col)

    # get correlation matrix
    matrix = Correlation.corr(stocks_df_vector, vector_col).collect()[0][0]
    corrmatrix = matrix.toArray().tolist()

    corrmatrix_df = spark.createDataFrame(corrmatrix,cols)

    for c in corrmatrix_df.columns:
        corrmatrix_df = corrmatrix_df.withColumn(c,F.round(c,2))

    stock_list = corrmatrix_df.columns #["AAPL","KO","NFLX","DIS","IBM","VZ","WMT","GE","TSLA","MA","AMZN","MSFT","UN","V"]

    # Create a DataFrame from the list of values
    values_df = spark.createDataFrame([Row(new_column=v) for v in stock_list])

    corrmatrix_df = corrmatrix_df.withColumn("row_index", monotonically_increasing_id())
    values_df = values_df.withColumn("row_index", monotonically_increasing_id())

    # Join the DataFrames on the row index
    corr_df = corrmatrix_df.join(values_df, on="row_index").drop("row_index")

    # Show the resulting DataFrame
    corr_df=corr_df.withColumnRenamed("new_column","stockName")
    return corr_df

In [7]:
corr_final_df = stocks_correlation(return_df,return_df.columns)
corr_final_df.show()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---------+
| AAPL| ABBV|  ABT| ADBE|  ADP| AMGN|  AMT| AMZN| AROW|ARTNA|  AXP|  AZN|   BA|BABA|  BAC| BCAL|  BHP| BKNG|  BTI|  CAT| CECO| CHEF|   CL| CMRX| COCO|  CRM| CRML| CSCO| CTOS|  CVX|  DEO|  DIS| ELLO| EQIX|  ETN|  FAT| GCMG|   GE|GOOGL|  GRC|   GS| GWRS| HIMX| HIPO|  HON|  IBM| ICHR| ISRG|  JNJ|  JPM|  KLG|  KMB|   KO|  LIN|  LLY|  LMT|   MA|  MCD| MDLZ| META|   MS| MSFT|  MXL| NFLX|  NKE|

In [8]:
def moderateCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.5 and 0.69".format(st)
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [9]:
def highCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.7 and 0.99".format(st)
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [10]:
moderated_correlation_portfolios = moderateCorrelation(corr_final_df,stock_lst)

In [11]:
moderated_correlation_portfolios

[['AAPL'],
 ['ABBV'],
 ['ABT'],
 ['CRM', 'ADBE'],
 ['ADP'],
 ['AMGN'],
 ['AMT'],
 ['GOOGL', 'MSFT', 'AMZN'],
 ['WFC', 'BAC', 'BCAL', 'AROW'],
 ['ARTNA'],
 ['CAT', 'JPM', 'GS', 'BKNG', 'MS', 'WFC', 'BAC', 'GE', 'AXP'],
 ['AZN'],
 ['BA'],
 ['NTES', 'BABA'],
 ['JPM', 'AXP', 'AROW', 'MS', 'WFC', 'BAC'],
 ['AROW', 'BCAL'],
 ['BHP'],
 ['AXP', 'ETN', 'BKNG'],
 ['KO', 'PM', 'BTI'],
 ['GS', 'AXP', 'IBM', 'ETN', 'PNNT', 'GRC', 'CAT'],
 ['CECO'],
 ['CHEF'],
 ['KO', 'PG', 'CL'],
 ['CMRX'],
 ['COCO'],
 ['ADBE', 'CRM'],
 ['CRML'],
 ['CSCO'],
 ['CTOS'],
 ['CVX'],
 ['DEO'],
 ['DIS'],
 ['ELLO'],
 ['EQIX'],
 ['CAT', 'NVDA', 'TSM', 'BKNG', 'ICHR', 'MSFT', 'GE', 'ETN'],
 ['FAT'],
 ['GCMG'],
 ['SPOT', 'AXP', 'ETN', 'GE'],
 ['AMZN', 'MSFT', 'GOOGL'],
 ['CAT', 'IBM', 'PNNT', 'GRC'],
 ['CAT', 'JPM', 'AXP', 'WFC', 'GS'],
 ['GWRS'],
 ['HIMX'],
 ['HIPO'],
 ['HON'],
 ['CAT', 'GRC', 'IBM'],
 ['TSM', 'ETN', 'ICHR'],
 ['TSM', 'META', 'ISRG'],
 ['KO', 'PFE', 'JNJ'],
 ['GS', 'AXP', 'WFC', 'BAC', 'JPM'],
 ['KLG'],
 ['P

In [14]:
high_correlation_portfolios = highCorrelation(corr_final_df,stock_lst)

In [15]:
high_correlation_portfolios

[['AAPL'],
 ['ABBV'],
 ['ABT'],
 ['ADBE'],
 ['ADP'],
 ['AMGN'],
 ['AMT'],
 ['AMZN'],
 ['AROW'],
 ['ARTNA'],
 ['AXP'],
 ['AZN'],
 ['BA'],
 ['BABA'],
 ['GS', 'BAC'],
 ['BCAL'],
 ['RIO', 'BHP'],
 ['BKNG'],
 ['BTI'],
 ['CAT'],
 ['CECO'],
 ['CHEF'],
 ['CL'],
 ['CMRX'],
 ['COCO'],
 ['CRM'],
 ['CRML'],
 ['CSCO'],
 ['CTOS'],
 ['CVX'],
 ['DEO'],
 ['DIS'],
 ['ELLO'],
 ['EQIX'],
 ['ETN'],
 ['FAT'],
 ['GCMG'],
 ['GE'],
 ['GOOGL'],
 ['GRC'],
 ['BAC', 'MS', 'GS'],
 ['GWRS'],
 ['HIMX'],
 ['HIPO'],
 ['HON'],
 ['IBM'],
 ['ICHR'],
 ['ISRG'],
 ['JNJ'],
 ['JPM'],
 ['KLG'],
 ['KMB'],
 ['KO'],
 ['LIN'],
 ['LLY'],
 ['LMT'],
 ['V', 'MA'],
 ['MCD'],
 ['MDLZ'],
 ['META'],
 ['GS', 'MS'],
 ['MSFT'],
 ['MXL'],
 ['NFLX'],
 ['NKE'],
 ['NSPR'],
 ['NTES'],
 ['NVDA'],
 ['ORCL'],
 ['PCYO'],
 ['PDD'],
 ['PFE'],
 ['PG'],
 ['PLCE'],
 ['PM'],
 ['PNNT'],
 ['PNST'],
 ['BHP', 'RIO'],
 ['SAP'],
 ['SIRI'],
 ['SNY'],
 ['SPOT'],
 ['SRG'],
 ['SUP'],
 ['TCI'],
 ['TH'],
 ['TJX'],
 ['TM'],
 ['TSLA'],
 ['TSM'],
 ['UAN'],
 ['UL'],
 ['UN

In [12]:
portfolios = []
#for port in high_correlation_portfolios:
#    if 5 <= len(port) <= 15:
#       portfolios.append(port)
for port in moderated_correlation_portfolios:
    if 5 < len(port) <= 15:
        portfolios.append(port)

In [13]:
# Extract a list of all the tickers in the dataframe
#tickers = list(return_df.columns)
#tickers

In [14]:
# Create 10 portfolios with each ensuing portfolio comprising of ONE additional stock.
#portfolios = []

#for i in range(1, len(tickers) + 1):
 #   portfolios.append(tickers[0:i])

In [13]:
portfolios

[['CAT', 'JPM', 'GS', 'BKNG', 'MS', 'WFC', 'BAC', 'GE', 'AXP'],
 ['JPM', 'AXP', 'AROW', 'MS', 'WFC', 'BAC'],
 ['GS', 'AXP', 'IBM', 'ETN', 'PNNT', 'GRC', 'CAT'],
 ['CAT', 'NVDA', 'TSM', 'BKNG', 'ICHR', 'MSFT', 'GE', 'ETN'],
 ['CL', 'JNJ', 'MDLZ', 'PM', 'BTI', 'KO'],
 ['AMZN', 'ORCL', 'GOOGL', 'ETN', 'META', 'MSFT'],
 ['NVDA', 'ORCL', 'ICHR', 'SAP', 'ETN', 'ISRG', 'TSM'],
 ['JPM', 'GS', 'AXP', 'AROW', 'MS', 'BAC', 'WFC']]

In [14]:
port_df = spark.createDataFrame([(row,) for row in portfolios], ["values"])
port_df_exploded = port_df.select(explode(col("values")).alias("value"))
unique_port_df = port_df_exploded.distinct()
unique_port =unique_port_df.rdd.flatMap(lambda x :x).collect()

In [15]:
unique_port

['AXP',
 'PM',
 'TSM',
 'PNNT',
 'JPM',
 'WFC',
 'META',
 'ICHR',
 'CL',
 'IBM',
 'JNJ',
 'ISRG',
 'MS',
 'GOOGL',
 'BAC',
 'ETN',
 'AROW',
 'GE',
 'CAT',
 'ORCL',
 'GS',
 'GRC',
 'SAP',
 'BTI',
 'AMZN',
 'MSFT',
 'MDLZ',
 'BKNG',
 'KO',
 'NVDA']

In [16]:
# Names to Portfolios
names = []

for i in range(1, len(portfolios)+1):
    names.append('Portfolio'+str(i))


In [17]:
names

['Portfolio1',
 'Portfolio2',
 'Portfolio3',
 'Portfolio4',
 'Portfolio5',
 'Portfolio6',
 'Portfolio7',
 'Portfolio8']

In [18]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
from pyspark.mllib.linalg import DenseMatrix

"""
Returns the Annualised Expected Return of a portfolio.
Annualises the return using the 'crude' method.
"""
def getPortReturns():
    portfolio_returns = []
    for port,weight in zip(portfolios,portfolio_weightage):
           
        num_stocks=len(port)
        # creating weight matrix  
        weight_matrix= DenseMatrix(1,num_stocks, weight)
        # Creating Row matrix for the Dense matrix to do dot_product using multiply function
        weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
        # Extracting mean return of the portfolio
        return_df_mean_list1=[]
        return_df_mean_list=[]
        for p in port:
            return_df_mean_list = return_df_mean.filter(return_df_mean.Stocks == p).select("Mean").rdd.flatMap(lambda x: x).collect()
            return_df_mean_list1.append(return_df_mean_list[0])
                
        return_df_matrix=DenseMatrix(len(return_df_mean_list1),1,return_df_mean_list1)
        # calculating dot_product of weight and mean return to get the expected return
        exp_ret_portfolio=weight_row_mat.multiply(return_df_matrix).rows.collect()
        # annual expected return through crude method
        exp_ret_portfolio_annual = exp_ret_portfolio[0][0] * 250
        # Returning the lsit of expected return of portfolios
        portfolio_returns.append(exp_ret_portfolio_annual)
    
    return portfolio_returns
    
    
    
    

In [19]:
# Calculate the risk of each portfolio
from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
import numpy as np
# arguments are weight and return_df of portfolios #df_temp = return_df[port]
def getPortRisk(weights,df_temp):
    portfolio_risks = []
    
    # calculating the variance covariance matrix of return_df of portfolio
    
    return_rdd = df_temp.rdd.map(list)
    return_row_mat = RowMatrix(return_rdd)
    vcv_matrix=return_row_mat.computeCovariance()
   
    # Variance of the portfolio = weight matrix dot_product variance-covariance matricx dot_product weight transpose matrix
    weight_matrix= DenseMatrix(1, num_stocks, weights) # weight matrix
    weight_matrix_t= DenseMatrix(num_stocks, 1, weights) # weight matrix Transpose
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    vcv_w=weight_row_mat.multiply(vcv_matrix) # dot_product of weight matrix and vcv matrix
    var_portfolio = vcv_w.multiply(weight_matrix_t).rows.collect() # dot_product of (weight matrix and vcv matrix in prev line) with weight transpose
    
    # calculating standard deviation from variance and compute annual sd of the portfolio
    var_portfolio=var_portfolio[0][0]
    sd_portfolio=np.sqrt(var_portfolio)
    sd_portfolio_annual = sd_portfolio * np.sqrt(250)
    
    # returning the portfolio risk i.e. standard deviation computed annually
    portfolio_risks.append(sd_portfolio_annual)
    
    return portfolio_risks

In [20]:
return_df_unique=return_df[unique_port]

In [21]:
return_df_unique.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|      AXP|       PM|      TSM|     PNNT|      JPM|      WFC|     META|     ICHR|       CL|      IBM|      JNJ|     ISRG|       MS|    GOOGL|      BAC|      ETN|     AROW|       GE|      CAT|     ORCL|       GS|      GRC|      SAP|      BTI|     AMZN|     MSFT|     MDLZ|     BKNG|       KO|     NVDA|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.002633| 0.004055| 0.003584| 0.010819| 0.004972| 0.009305|-0.004324|-0.029048| 0.002939| 0.

In [22]:
# Calculating the weightage of the stocks for the portfolio by minimising risks
from scipy.optimize import minimize
import numpy as np

portfolio_weightage=[]
for port in portfolios:

    num_stocks = len(return_df_unique[port].columns)  # being the number of stocks (this is a 'global' variable)
    init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0) [Intially assume equal weightge]
    args=return_df_unique[port] # second argument of getPortRisk() function
    bounds = tuple((0, 1) for i in range(num_stocks)) # weightage will be between 0 and 1
    cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1}) # sum of weightage will be 1
    # getting the optimised weightage through minimise() function with getPortRisk() as objective function and applying bounadries and constraints as adefined above
    results = minimize(fun=getPortRisk, x0=init_weights,args=args, bounds=bounds, constraints=cons)
    # converting the values to float and rounding off to 2 decimal places
    rounded_result = np.round(results.x, decimals=2)
    print(rounded_result)
    temp_rdd=sc.parallelize(rounded_result)
    temp_rdd=temp_rdd.map(lambda x:float(x))
    
    portfolio_weightage.append(temp_rdd.collect())


[0.15 0.21 0.   0.09 0.14 0.11 0.15 0.02 0.14]
[0.18 0.33 0.11 0.16 0.05 0.17]
[0.06 0.18 0.26 0.04 0.47 0.   0.  ]
[0.28 0.   0.   0.05 0.01 0.63 0.04 0.  ]
[0.29 0.05 0.04 0.15 0.08 0.4 ]
[0.12 0.   0.1  0.08 0.   0.7 ]
[0.   0.01 0.   0.31 0.12 0.56 0.  ]
[0.18 0.   0.33 0.11 0.16 0.17 0.05]


In [23]:
portfolio_weightage

[[0.15, 0.21, 0.0, 0.09, 0.14, 0.11, 0.15, 0.02, 0.14],
 [0.18, 0.33, 0.11, 0.16, 0.05, 0.17],
 [0.06, 0.18, 0.26, 0.04, 0.47, 0.0, 0.0],
 [0.28, 0.0, 0.0, 0.05, 0.01, 0.63, 0.04, 0.0],
 [0.29, 0.05, 0.04, 0.15, 0.08, 0.4],
 [0.12, 0.0, 0.1, 0.08, 0.0, 0.7],
 [0.0, 0.01, 0.0, 0.31, 0.12, 0.56, 0.0],
 [0.18, 0.0, 0.33, 0.11, 0.16, 0.17, 0.05]]

In [24]:
# Calculating the investment amount of the portfolio i.e. latest closing price

latest_date = df.select(max("date").alias("latest_date")).collect()[0]["latest_date"]
portfolio_amount = []

for port in portfolios:
    amount_rdd=df.filter(df.date == latest_date).select(port).rdd.flatMap(lambda x: x).map(lambda x: float(x))
    total_amount= amount_rdd.collect()
    portfolio_amount.append(total_amount)
    
    

In [25]:
print(latest_date)

2024-08-09


In [26]:
portfolio_amount

[[338.26, 205.8, 490.26, 3443.05, 94.72, 52.76, 38.28, 167.13, 237.85],
 [205.8, 237.85, 27.64, 94.72, 52.76, 38.28],
 [490.26, 237.85, 191.45, 291.64, 6.93, 37.05, 338.26],
 [338.26, 104.75, 167.12, 3443.05, 28.9, 406.02, 167.13, 291.64],
 [102.07, 160.62, 69.72, 116.01, 35.96, 68.68],
 [166.94, 132.06, 163.67, 291.64, 517.77, 406.02],
 [104.75, 132.06, 28.9, 208.53, 291.64, 463.425, 167.12],
 [205.8, 490.26, 237.85, 27.64, 94.72, 38.28, 52.76]]

In [27]:


# calculating mean return for each stocks and creating a dtaframe
data1=unique_port
data2=[]
for c in unique_port:
    data2.append(return_df.select(mean(col(c))).collect()[0][0]) 
data = list(zip(data1, data2))
return_df_mean=spark.createDataFrame(data,["Stocks","Mean"])
return_df_mean = return_df_mean.withColumn("Mean", round(return_df_mean["Mean"], 6))
return_df_mean.show()

+------+---------+
|Stocks|     Mean|
+------+---------+
|   AXP|  2.51E-4|
|    PM| 0.002354|
|   TSM| 0.003133|
|  PNNT| 0.001486|
|   JPM|  9.15E-4|
|   WFC| -5.25E-4|
|  META| 0.001559|
|  ICHR|-0.003554|
|    CL| 0.001805|
|   IBM|  6.99E-4|
|   JNJ|  2.01E-4|
|  ISRG| 0.001286|
|    MS| 0.001995|
| GOOGL| 0.002007|
|   BAC|  9.62E-4|
|   ETN|  6.06E-4|
|  AROW|  1.24E-4|
|    GE| 0.002767|
|   CAT|  -1.4E-5|
|  ORCL| 0.002532|
+------+---------+
only showing top 20 rows



In [28]:
# function call for returns calulation as per thw portflio weightage
portfolio_return_wt=getPortReturns()


In [29]:
portfolio_return_wt

[0.138735,
 0.179415,
 0.271445,
 0.15205250000000003,
 0.40145,
 0.24579,
 0.31832000000000005,
 0.17941499999999996]

In [37]:
print(return_df_sp500.count())
print(return_df_unique.count())

102
102


In [39]:
return_df_sp500 = return_df_sp500.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
return_df_unique= return_df_unique.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

return_df_sp500=return_df_unique.join(return_df_sp500,"row_idx").drop("row_idx")
return_df_unique=return_df_unique.drop("row_idx")
#return_df_sp500.show()

In [58]:
# Calculating beta_value with Linear Regression 

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=["SP500"], outputCol="features")
data_with_features = assembler.transform(return_df_sp500)
betaValues=[]
for col in return_df_unique.columns:
# Create a LinearRegression model and fit it to the data
    lr = LinearRegression(featuresCol="features", labelCol=col) # passing each stock to LR as labelCol
    model = lr.fit(data_with_features)
    betaValues.append(model.coefficients[0]) # model.coefficients[0] gives beta value of specific stock 

print(betaValues)


[0.032312223105690256, 0.033227364735305805, 0.0921652118974647, 0.001845983626216809, 0.040896723204248635, 0.004566957270287947, 0.04679536220728478, -0.004067589267233715, 0.022215174612057943, -0.0037754139237596318, -0.005229437655726407, 0.1408815732327109, 0.027461773008582515, 0.07177771602946303, 0.012003947456860042, 0.02714578314879822, 0.01010823313651886, 0.04266870400092122, -0.024914409062894632, 0.05719520060530078, 0.16992531089637936, 0.005903346257912311, 0.04182181902060536, 0.008305038117319826, 0.028223230567677955, 0.0911011495678594, -0.008820729860444973, 0.9926446432959586, 0.010319465884419713, 0.09547402939749257]


In [59]:
#Creating dataframe for Stocks with beta value
d1=unique_port
d2=[float(x) for x in betaValues]
d=zip(d1,d2)
beta_df=spark.createDataFrame(d,["Stocks","Beta"])
beta_df=beta_df.withColumn("Beta",round("Beta",2))
beta_df.show()

+------+-----+
|Stocks| Beta|
+------+-----+
|   AXP| 0.03|
|    PM| 0.03|
|   TSM| 0.09|
|  PNNT|  0.0|
|   JPM| 0.04|
|   WFC|  0.0|
|  META| 0.05|
|  ICHR|  0.0|
|    CL| 0.02|
|   IBM|  0.0|
|   JNJ|-0.01|
|  ISRG| 0.14|
|    MS| 0.03|
| GOOGL| 0.07|
|   BAC| 0.01|
|   ETN| 0.03|
|  AROW| 0.01|
|    GE| 0.04|
|   CAT|-0.02|
|  ORCL| 0.06|
+------+-----+
only showing top 20 rows



In [60]:
# Grouping the beta values for the portfolio

beta_port=[]
for port in portfolios:
    beta_list=[]
    for p in port:
        beta_rdd=beta_df.filter(beta_df.Stocks == p).select("Beta").rdd.flatMap(lambda x: x).map(lambda x: float(x))
        beta_list.append(beta_rdd.collect())
        beta_list_rdd=sc.parallelize(beta_list)
        beta_list_f = beta_list_rdd.flatMap(lambda x: x).collect()
    beta_port.append(beta_list_f)
        
print(beta_port)

[[-0.02, 0.04, 0.17, 0.99, 0.03, 0.0, 0.01, 0.04, 0.03], [0.04, 0.03, 0.01, 0.03, 0.0, 0.01], [0.17, 0.03, 0.0, 0.03, 0.0, 0.01, -0.02], [-0.02, 0.1, 0.09, 0.99, 0.0, 0.09, 0.04, 0.03], [0.02, -0.01, -0.01, 0.03, 0.01, 0.01], [0.03, 0.06, 0.07, 0.03, 0.05, 0.09], [0.1, 0.06, 0.0, 0.04, 0.03, 0.14, 0.09], [0.04, 0.17, 0.03, 0.01, 0.03, 0.01, 0.0]]


In [76]:
# CAlculating the final Portfolio Dataframe incorporating Names, Portfolios, Weightage,return,Amount,Beta values

portfolio_return_wt=[float(x) for x in portfolio_return_wt]

data=list(zip(names,portfolios,portfolio_weightage,portfolio_return_wt,portfolio_amount,beta_port))
schema = StructType([
    StructField("Names", StringType(), True),
    StructField("Portfolios", ArrayType(StringType()), True),
    StructField("Weightage", ArrayType(FloatType()), True),
    StructField("TotalReturns", FloatType(), True),
    StructField("Amount", ArrayType(FloatType()), True),
    StructField("BetaPort", ArrayType(FloatType()), True)])
                
final_portfolio_df = spark.createDataFrame(data, schema)

final_portfolio_df.show()

+----------+--------------------+--------------------+------------+--------------------+--------------------+
|     Names|          Portfolios|           Weightage|TotalReturns|              Amount|            BetaPort|
+----------+--------------------+--------------------+------------+--------------------+--------------------+
|Portfolio1|[CAT, JPM, GS, BK...|[0.15, 0.21, 0.0,...|    0.138735|[338.26, 205.8, 4...|[-0.02, 0.04, 0.1...|
|Portfolio2|[JPM, AXP, AROW, ...|[0.18, 0.33, 0.11...|    0.179415|[205.8, 237.85, 2...|[0.04, 0.03, 0.01...|
|Portfolio3|[GS, AXP, IBM, ET...|[0.06, 0.18, 0.26...|    0.271445|[490.26, 237.85, ...|[0.17, 0.03, 0.0,...|
|Portfolio4|[CAT, NVDA, TSM, ...|[0.28, 0.0, 0.0, ...|   0.1520525|[338.26, 104.75, ...|[-0.02, 0.1, 0.09...|
|Portfolio5|[CL, JNJ, MDLZ, P...|[0.29, 0.05, 0.04...|     0.40145|[102.07, 160.62, ...|[0.02, -0.01, -0....|
|Portfolio6|[AMZN, ORCL, GOOG...|[0.12, 0.0, 0.1, ...|     0.24579|[166.94, 132.06, ...|[0.03, 0.06, 0.07...|
|Portfolio

In [77]:
# CAlculating the Investment amount and beta value as per the  weightage

final_portfolio_df = final_portfolio_df.withColumn("InvestmentAmount", expr("aggregate(zip_with(Weightage, Amount, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("Amount")

final_portfolio_df = final_portfolio_df.withColumn("PortfolioBeta", expr("aggregate(zip_with(Weightage, BetaPort, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("BetaPort")

final_portfolio_df = final_portfolio_df.withColumn("TotalReturns", round(final_portfolio_df["TotalReturns"], 2)) \
.withColumn("InvestmentAmount", round(final_portfolio_df["InvestmentAmount"], 2)) \
.withColumn("PortfolioBeta", round(final_portfolio_df["PortfolioBeta"], 2))

final_portfolio_df.show(truncate=False)

+----------+-------------------------------------------+-----------------------------------------------------+------------+----------------+-------------+
|Names     |Portfolios                                 |Weightage                                            |TotalReturns|InvestmentAmount|PortfolioBeta|
+----------+-------------------------------------------+-----------------------------------------------------+------------+----------------+-------------+
|Portfolio1|[CAT, JPM, GS, BKNG, MS, WFC, BAC, GE, AXP]|[0.15, 0.21, 0.0, 0.09, 0.14, 0.11, 0.15, 0.02, 0.14]|0.14        |465.28          |0.11         |
|Portfolio2|[JPM, AXP, AROW, MS, WFC, BAC]             |[0.18, 0.33, 0.11, 0.16, 0.05, 0.17]                 |0.18        |142.88          |0.02         |
|Portfolio3|[GS, AXP, IBM, ETN, PNNT, GRC, CAT]        |[0.06, 0.18, 0.26, 0.04, 0.47, 0.0, 0.0]             |0.27        |136.93          |0.02         |
|Portfolio4|[CAT, NVDA, TSM, BKNG, ICHR, MSFT, GE, ETN]|[0.28, 0.0, 0.

In [78]:
# Incorporate Volatility based on beta value , Beta VAlue > 1 High Volatile else Less Volatile

from pyspark.sql import functions as F

final_portfolio_df = final_portfolio_df.withColumn("Volatility", F.when(final_portfolio_df["PortfolioBeta"] > 1, "High") \
                                                   .otherwise("Low"))
final_portfolio_df=final_portfolio_df.drop("PortfolioBeta")
final_portfolio_df.show()




+----------+--------------------+--------------------+------------+----------------+----------+
|     Names|          Portfolios|           Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+--------------------+--------------------+------------+----------------+----------+
|Portfolio1|[CAT, JPM, GS, BK...|[0.15, 0.21, 0.0,...|        0.14|          465.28|       Low|
|Portfolio2|[JPM, AXP, AROW, ...|[0.18, 0.33, 0.11...|        0.18|          142.88|       Low|
|Portfolio3|[GS, AXP, IBM, ET...|[0.06, 0.18, 0.26...|        0.27|          136.93|       Low|
|Portfolio4|[CAT, NVDA, TSM, ...|[0.28, 0.0, 0.0, ...|        0.15|          529.63|       Low|
|Portfolio5|[CL, JNJ, MDLZ, P...|[0.29, 0.05, 0.04...|         0.4|           88.17|       Low|
|Portfolio6|[AMZN, ORCL, GOOG...|[0.12, 0.0, 0.1, ...|        0.25|          343.94|       Low|
|Portfolio7|[NVDA, ORCL, ICHR...|[0.0, 0.01, 0.0, ...|        0.32|          360.48|       Low|
|Portfolio8|[JPM, GS, AXP, AR...|[0.18, 

In [79]:
df_exploded1 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility",posexplode("Portfolios").alias("pos", "Portfolios"))
df_exploded2 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility", posexplode("Weightage").alias("pos", "Weightage"))

df_final = df_exploded1.join(df_exploded2, on=["Names","TotalReturns","InvestmentAmount","Volatility", "pos"]) \
.select("Names", "Portfolios", "Weightage","TotalReturns","InvestmentAmount","Volatility")

df_sorted = df_final.sort("Names")
df_sorted.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|        MS|     0.14|        0.14|          465.28|       Low|
|Portfolio1|       WFC|     0.11|        0.14|          465.28|       Low|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|
|Portfolio1|       CAT|     0.15|        0.14|          465.28|       Low|
|Portfolio1|       JPM|     0.21|        0.14|          465.28|       Low|
|Portfolio1|        GS|      0.0|        0.14|          465.28|       Low|
|Portfolio1|        GE|     0.02|        0.14|          465.28|       Low|
|Portfolio1|       BAC|     0.15|        0.14|          465.28|       Low|
|Portfolio1|       AXP|     0.14|        0.14|          465.28|       Low|
|Portfolio2|      AROW|     0.11|        0.18|          142.88|       Low|
|Portfolio2|       BAC|  

In [80]:
df_sorted_fin=df_sorted.filter(df_sorted.Weightage != 0)

In [81]:
df_sorted_fin.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|        MS|     0.14|        0.14|          465.28|       Low|
|Portfolio1|       JPM|     0.21|        0.14|          465.28|       Low|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|
|Portfolio1|       AXP|     0.14|        0.14|          465.28|       Low|
|Portfolio1|       WFC|     0.11|        0.14|          465.28|       Low|
|Portfolio1|        GE|     0.02|        0.14|          465.28|       Low|
|Portfolio1|       CAT|     0.15|        0.14|          465.28|       Low|
|Portfolio1|       BAC|     0.15|        0.14|          465.28|       Low|
|Portfolio2|       JPM|     0.18|        0.18|          142.88|       Low|
|Portfolio2|       WFC|     0.05|        0.18|          142.88|       Low|
|Portfolio2|       AXP|  

In [82]:
filepath='file:////home/talentum/shared/Project/Project_codes/source/company_names.csv'

company_df=spark.read.option("header",True).csv(filepath)

In [83]:
company_df.show()

+------+--------------------+
|symbol|             company|
+------+--------------------+
|  AAPL|          Apple Inc.|
|  ABBV|         AbbVie Inc.|
|   ABT| Abbott Laboratories|
|  ADBE|          Adobe Inc.|
|   ADP|Automatic Data Pr...|
|  AMGN|          Amgen Inc.|
|   AMT|American Tower Co...|
|  AMZN|    Amazon.com, Inc.|
|  AROW|Arrow Financial C...|
| ARTNA|Artesian Resource...|
|   AXP|American Express ...|
|   AZN|     AstraZeneca PLC|
|    BA|  The Boeing Company|
|  BABA|Alibaba Group Hol...|
|   BAC|Bank of America C...|
|  BCAL|Southern Californ...|
|   BHP|   BHP Group Limited|
|  BKNG|Booking Holdings ...|
|   BTI|British American ...|
|   CAT|    Caterpillar Inc.|
+------+--------------------+
only showing top 20 rows



In [84]:
df_sorted_fin=df_sorted_fin.join(company_df,df_sorted_fin.Portfolios==company_df.symbol,"inner")
df_sorted_fin=df_sorted_fin.drop("symbol")
df_sorted_fin.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|       WFC|     0.11|        0.14|          465.28|       Low|Wells Fargo & Com...|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|Booking Holdings ...|
|Portfolio1|       JPM|     0.21|        0.14|          465.28|       Low|JPMorgan Chase & Co.|
|Portfolio1|       AXP|     0.14|        0.14|          465.28|       Low|American Express ...|
|Portfolio1|       CAT|     0.15|        0.14|          465.28|       Low|    Caterpillar Inc.|
|Portfolio1|        GE|     0.02|        0.14|          465.28|       Low|General Electric ...|
|Portfolio1|        MS|     0.14|        0.14|          465.28|       Low|      Morgan Stanley|
|Portfolio1|       BAC|     0.15|       

In [85]:
df_sorted_fin.write.mode("overwrite").saveAsTable("Portfolios")

In [86]:
portfoliodata=spark.sql("select * from Portfolios")

In [87]:
portfoliodata.show()
stockdata.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|       WFC|     0.11|        0.14|          465.28|       Low|Wells Fargo & Com...|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|Booking Holdings ...|
|Portfolio1|       JPM|     0.21|        0.14|          465.28|       Low|JPMorgan Chase & Co.|
|Portfolio1|       AXP|     0.14|        0.14|          465.28|       Low|American Express ...|
|Portfolio1|       CAT|     0.15|        0.14|          465.28|       Low|    Caterpillar Inc.|
|Portfolio1|        GE|     0.02|        0.14|          465.28|       Low|General Electric ...|
|Portfolio1|        MS|     0.14|        0.14|          465.28|       Low|      Morgan Stanley|
|Portfolio1|       BAC|     0.15|       

In [88]:
joineddf = portfoliodata.join(stockdata, portfoliodata["Portfolios"] == stockdata["symbol"], "inner")
joineddf=joineddf.drop("label")
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+-------+----------+-------+-------+-------+----------------+------+---------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose|change|changeOverTime|changePercent|  close|      date|   high|    low|   open|unadjustedVolume|volume|     vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+-------+----------+-------+-------+-------+----------------+------+---------+
|Portfolio4|      BKNG|     0.05|        0.15|          529.63|       Low|Booking Holdings ...|  BKNG| 3443.05|  48.2|        0.0142|         1.42|3443.05|2024-08-09| 3456.0|3394.85|3394.85|          251814|251814|3422.1875|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|Booking Holdings ...|  BKN

In [89]:
joineddf.write.mode("overwrite").saveAsTable("PortfoliosDetail")

In [90]:
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+-------+----------+-------+-------+-------+----------------+------+---------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose|change|changeOverTime|changePercent|  close|      date|   high|    low|   open|unadjustedVolume|volume|     vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+------+--------------+-------------+-------+----------+-------+-------+-------+----------------+------+---------+
|Portfolio4|      BKNG|     0.05|        0.15|          529.63|       Low|Booking Holdings ...|  BKNG| 3443.05|  48.2|        0.0142|         1.42|3443.05|2024-08-09| 3456.0|3394.85|3394.85|          251814|251814|3422.1875|
|Portfolio1|      BKNG|     0.09|        0.14|          465.28|       Low|Booking Holdings ...|  BKN

In [91]:
filepath='file:////home/talentum/csvfile'

#dffile=spark.sql("select * from PortfoliosDetail")
# Write DataFrame to CSV file with header
#sd.write.mode("overwrite").format("csv").save(filepath)

joineddf.coalesce(1).write.mode("overwrite").option("header","true").format("csv").save(filepath)